In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [ ]:
#load data
sciq = load_dataset("sciq")

quotes_az = pd.read_csv(r"H:\Curated-LLM-Data\data\quotes_by_author.csv",index_col=0)
quotes_gracious = pd.read_csv(r"H:\Curated-LLM-Data\data\graciousquotes.csv",index_col=0)
combined = pd.concat([quotes_az.dropna(),quotes_gracious.dropna()],axis=0).dropna()

In [24]:
sciq['train'][0]

{'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'distractor3': 'viruses',
 'distractor1': 'protozoa',
 'distractor2': 'gymnosperms',
 'correct_answer': 'mesophilic organisms',
 'support': 'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}

'What type of organism is commonly used in preparation of foods such as cheese and yogurt?'

'What type of organism is commonly used in preparation of foods such as cheese and yogurt?'

In [ ]:
# Step 1: Install the Transformers library
# !pip install transformers

# Step 2: Import required libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

# Step 3: Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Step 4: Create a custom dataset class
class QuotesDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, 'text']
        encoded_text = self.tokenizer.encode_plus(
            text, add_special_tokens=True, return_tensors='pt', truncation=True, max_length=1024
        )
        return encoded_text['input_ids'].squeeze()

# Step 5: Define a function to encode the text
def collate_fn(batch):
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)

# Step 6: Load the dataframe containing the text
import pandas as pd
data = {'text': ['Quote 1', 'Quote 2', 'Quote 3']}
quotes = pd.DataFrame(data)

# Step 7: Create a DataLoader
dataset = QuotesDataset(quotes, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# Example: Iterate over the DataLoader
for batch in dataloader:
    print(batch)
    

In [ ]:

# Step 1: Import required libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

# Continue from the previous example with the QuotesDataset class, tokenizer, and dataloader

# Step 2: Load the GPT-2 model and set it to the training mode
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.train()
model.to('cuda')

# Step 3: Define a loss function, optimizer, and learning rate
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader)*10)

# Step 4: Define the training loop
def train(dataloader, model, tokenizer, criterion, optimizer, scheduler, device='cuda'):
    model.train()
    for epoch in range(10):
        print(f"Epoch {epoch+1}")
        for step, batch in enumerate(dataloader):
            batch = batch.to(device)
            outputs = model(batch, labels=batch)
            loss = outputs[0]
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            if step % 10 == 0:
                print(f"Step {step} - Loss: {loss.item()}")
                
# Step 5: Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train(dataloader, model, tokenizer, criterion, optimizer, scheduler, device)